In [1]:
#pip install poke-env
#pip install --upgrade poke-env
#pip show poke-env
#cd "C:\Users\Isac\Desktop\Pokémon project\pythonProject\pokemon-showdown"
#node pokemon-showdown start --no-security

In [2]:
import numpy as np
from poke_env.teambuilder import Teambuilder

class RandomTeamFromPool(Teambuilder):
    def __init__(self, teams):
        self.teams = [self.join_team(self.parse_showdown_team(team)) for team in teams]

    def yield_team(self):
        return np.random.choice(self.teams)
    

In [ ]:
import os

relative_path = "pokémonTeams/teams1"

teamsPaths = []
for root, dirs, files in os.walk(relative_path):
	for file in files:
		teamsPaths.append(os.path.join(root,file))

teams = []
for teamPath in teamsPaths:
    with open(teamPath, "r") as team:
        teams.append(team.read())


In [4]:
print(teams)
custom_builder = RandomTeamFromPool(teams)

['Exeggutor @ Leftovers  \nAbility: Chlorophyll  \n- Explosion  \n- Psychic  \n- Stun Spore  \n- Giga Drain  \n\nCharizard @ Leftovers  \nAbility: Blaze  \n- Belly Drum  \n- Earthquake  \n- Fire Blast  \n- Rock Slide  \n\nRaikou @ Leftovers  \nAbility: Pressure  \nIVs: 26 Def  \n- Rest  \n- Hidden Power [Ice]  \n- Sleep Talk  \n- Thunder  \n\nRhydon @ Leftovers  \nAbility: Lightning Rod  \n- Rock Slide  \n- Zap Cannon  \n- Earthquake  \n- Roar  \n\nSnorlax @ Leftovers  \nAbility: Immunity  \n- Double-Edge  \n- Earthquake  \n- Lovely Kiss  \n- Self-Destruct  \n\nStarmie @ Leftovers  \nAbility: Illuminate  \n- Rapid Spin  \n- Recover  \n- Surf  \n- Thunder Wave  ', 'Snorlax @ Leftovers  \nAbility: No Ability  \n- Double-Edge  \n- Earthquake  \n- Flamethrower  \n- Rest  \n\nCloyster @ Leftovers  \nAbility: No Ability  \n- Surf  \n- Toxic  \n- Spikes  \n- Rest  \n\nZapdos @ Leftovers  \nAbility: No Ability  \nIVs: 26 Def  \n- Thunder  \n- Hidden Power [Ice]  \n- Rest  \n- Sleep Talk  \n\nG

In [5]:
import subprocess
import json

def run_javascript(file_path, *args):
    result = subprocess.run(["node", file_path] + list(args), capture_output=True, text=True, check=True)
    return json.loads(result.stdout)


In [6]:
from poke_env.player.player import Player
from poke_env.environment.abstract_battle import AbstractBattle
import requests

# Define a custom player by subclassing Player
class CustomPlayer(Player):
    def choose_move(self, battle: AbstractBattle):
        if battle.available_moves:
            return self.create_order(max(battle.available_moves, key=lambda move: move.base_power*move.accuracy))
        else:
            return self.choose_random_move(battle)

# Define a optimized player by subclassing Player
class OptimizedPlayer(Player):                
    def choose_move(self, battle: AbstractBattle):
        # print(f"Own pokemon: {battle.active_pokemon}")
        # print(f"Own pokemon name: {battle.active_pokemon.species}")
        # print(f"Own pokemon level: {battle.active_pokemon.level}")
        # print(f"Own pokemon item: {battle.active_pokemon.item}")
        # print(f"Own pokemon status: {battle.active_pokemon.status}")
        # print(f"Own pokemon ability: {battle.active_pokemon.ability}")
        # print(f"Own pokemon boosts: {battle.active_pokemon.boosts}")
        
        # print(f"Own pokemon stats: {battle.active_pokemon.stats}")
        # print(f"Own pokemon base stats: {battle.active_pokemon.base_stats}")
        
        # print(f"Opponent pokemon: {battle.opponent_active_pokemon}")
        # print(f"Opponent pokemon name: {battle.opponent_active_pokemon.species}")
        
        # print()
        if battle.available_moves:
            attacker = battle.active_pokemon
            defender = battle.opponent_active_pokemon

            params = {
                'attackerName': str(attacker.species),
                'attackerLevel': str(attacker.level),
                'attackerItem': str(attacker.item),
                'attackerStatus':str(attacker.status),
                'attackerAbility':str(attacker.ability),
                'attackerBoosts':str(attacker.boosts),
                'defenderName': str(defender.species),
                'defenderLevel': str(defender.level),
                'defenderItem':  str(defender.item),
                'defenderStatus': str(defender.status),
                'defenderAbility': str(defender.ability),
                'defenderBoosts': str(defender.boosts),
                'moveName': None
            }
            
            average_damage_moves = {}
            for current_move in battle.available_moves:
            
                params['moveName'] = str(current_move.id)
                
                response = requests.get('http://localhost:3000/calculate-damage', params=params)
                calculate_damage = response.json()
                
                # print(f"Current moves: {current_move}")
                # print(f"Current move damage: {calculate_damage}")
                
                if calculate_damage == 0:
                    average_damage_moves[current_move] = 0
                else:
                    average_damage_moves[current_move] = sum(calculate_damage)/len(calculate_damage)
                    
            # print(f"Current moves: {average_damage_moves}")
            # print(f"Picked move: {max(average_damage_moves, key=average_damage_moves.get)}")
            return self.create_order(max(average_damage_moves, key=average_damage_moves.get))
        else:
            return self.choose_random_move(battle)



In [7]:
from poke_env import RandomPlayer
from poke_env import LocalhostServerConfiguration

server_configuration = LocalhostServerConfiguration
generation = "gen2ou"

optimized_player = OptimizedPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

custom_player = CustomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

random_player = RandomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

random_player2 = RandomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)


In [ ]:
import time
start_time = time.time()
await optimized_player.battle_against(custom_player, n_battles=1)
print("--- %s seconds ---" % (time.time() - start_time))

2024-12-03 13:37:56,694 - OptimizedPlayer 1 - ERROR - Unhandled exception raised while handling message:
>battle-gen2ou-901
|player|p2|CustomPlayer 1|1|
|teamsize|p1|6
|teamsize|p2|6
|gen|2
|tier|[Gen 2] OU
|rule|Sleep Clause Mod: Limit one foe put to sleep
|rule|Freeze Clause Mod: Limit one foe frozen
|rule|Species Clause: Limit one of each Pokémon
|rule|OHKO Clause: OHKO moves are banned
|rule|Evasion Items Clause: Evasion items are banned
|rule|Evasion Moves Clause: Evasion moves are banned
|rule|Endless Battle Clause: Forcing endless battles is banned
|rule|HP Percentage Mod: HP is shown in percentages
|
|t:|1733229476
|start
|switch|p1a: Exeggutor|Exeggutor, M|393/393
|switch|p2a: Zapdos|Zapdos|100/100
|turn|1
Traceback (most recent call last):
  File "/home/agus/Documents/ITBA/POLIMI/Multidisciplinary Project/Pokemon-Strategy-using-Reinforcement-Learning/env/lib/python3.10/site-packages/requests/models.py", line 974, in json
    return complexjson.loads(self.text, **kwargs)
  Fil

In [ ]:
print(f"Battles played: {optimized_player.n_finished_battles}")
print(f"Battles won: {optimized_player.n_won_battles}")
print(f"Win rate: {optimized_player.win_rate}")

Battles played: 10
Battles won: 8
Win rate: 0.8


In [ ]:
import pandas as pd

# Define the columns for the DataFrame
# columns = ['battle_number', 'battle_turn', 'side_conditions', 'opponent_side_conditions', 'weather', 'fields', 'active_pokemon', 'opponent_active_pokemon', 'events']
columns = ['battle_number', 'battle_turn', 'active_pokemon', 'opponent_active_pokemon', 'events']


def parse_battle_observations():
    battle_results = []
    for battle_number, battle in enumerate(optimized_player.battles.values(), start=1):
        observations = battle.observations
        # print(observations)

        for turn, observation in observations.items():
            # side_conditions = observation.side_conditions
            # opponent_side_conditions = observation.opponent_side_conditions
            # weather = observation.weather
            # fields = observation.fields
            active_pokemon = observation.active_pokemon.species if observation.active_pokemon else None
            opponent_active_pokemon = observation.opponent_active_pokemon.species if observation.opponent_active_pokemon else None
            events = observation.events

            # Clean the events from redundant information
            for event in events:
                if event[0] == "":
                    event.pop(0)

            if events[-1][0] == "turn":
                events.pop(-1)
            if events[-1][0] == "upkeep":
                events.pop(-1)

                

            battle_results.append({
                'battle_number': battle_number,
                'battle_turn': turn,
                # 'side_conditions': side_conditions,
                # 'opponent_side_conditions': opponent_side_conditions,
                # 'weather': weather,
                # 'fields': fields,
                'active_pokemon': active_pokemon,
                'opponent_active_pokemon': opponent_active_pokemon,
                'events': events
            })
    return battle_results

# Parse the battle observations and create a DataFrame
battle_results = parse_battle_observations()
df = pd.DataFrame(battle_results, columns=columns)

# Export DataFrame to JSON
df.to_json('results/battle_results.json', orient='records')
print(df)



   battle_number  battle_turn active_pokemon opponent_active_pokemon  \
0              1            0           None                    None   
1              1            1         zapdos               exeggutor   
2              1            2      exeggutor                  rhydon   
3              1            3      exeggutor                  raikou   
4              1            4       cloyster               charizard   
5              1            5        snorlax                 snorlax   
6              1            6         gengar                 starmie   
7              1            7       alakazam                 starmie   

                                              events  
0  [[init, battle], [title, OptimizedPlayer 3 vs....  
1  [[move, p1a: Zapdos, Hidden Power, p2a: Exeggu...  
2  [[move, p1a: Exeggutor, Giga Drain, p2a: Rhydo...  
3  [[move, p2a: Raikou, Thunder, p1a: Exeggutor],...  
4  [[move, p2a: Charizard, Fire Blast, p1a: Cloys...  
5  [[move, p1a: Snorl

In [ ]:
import asyncio

async def main():
    await optimized_player.battle_against(custom_player, n_battles=1)

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running

In [ ]:
import itertools

# Define natures and their stat changes
natures = {
    'Hardy': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Lonely': {'atk': 1.1, 'def': 0.9, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Brave': {'atk': 1.1, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 0.9},
    'Adamant': {'atk': 1.1, 'def': 1.0, 'spa': 0.9, 'spd': 1.0, 'spe': 1.0},
    'Naughty': {'atk': 1.1, 'def': 1.0, 'spa': 1.0, 'spd': 0.9, 'spe': 1.0},
    'Bold': {'atk': 0.9, 'def': 1.1, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Docile': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Relaxed': {'atk': 1.0, 'def': 1.1, 'spa': 1.0, 'spd': 1.0, 'spe': 0.9},
    'Impish': {'atk': 1.0, 'def': 1.1, 'spa': 0.9, 'spd': 1.0, 'spe': 1.0},
    'Lax': {'atk': 1.0, 'def': 1.1, 'spa': 1.0, 'spd': 0.9, 'spe': 1.0},
    'Timid': {'atk': 0.9, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.1},
    'Hasty': {'atk': 1.0, 'def': 0.9, 'spa': 1.0, 'spd': 1.0, 'spe': 1.1},
    'Serious': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Jolly': {'atk': 1.0, 'def': 1.0, 'spa': 0.9, 'spd': 1.0, 'spe': 1.1},
    'Naive': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 0.9, 'spe': 1.1},
    'Modest': {'atk': 0.9, 'def': 1.0, 'spa': 1.1, 'spd': 1.0, 'spe': 1.0},
    'Mild': {'atk': 1.0, 'def': 0.9, 'spa': 1.1, 'spd': 1.0, 'spe': 1.0},
    'Quiet': {'atk': 1.0, 'def': 1.0, 'spa': 1.1, 'spd': 1.0, 'spe': 0.9},
    'Bashful': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Rash': {'atk': 1.0, 'def': 1.0, 'spa': 1.1, 'spd': 0.9, 'spe': 1.0},
    'Calm': {'atk': 0.9, 'def': 1.0, 'spa': 1.0, 'spd': 1.1, 'spe': 1.0},
    'Gentle': {'atk': 1.0, 'def': 0.9, 'spa': 1.0, 'spd': 1.1, 'spe': 1.0},
    'Sassy': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.1, 'spe': 0.9},
    'Careful': {'atk': 1.0, 'def': 1.0, 'spa': 0.9, 'spd': 1.1, 'spe': 1.0},
    'Quirky': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0}
}

# Function to calculate a stat
def calculate_stat(base_stat, iv, ev, level, nature_modifier):
    return int((((2 * base_stat + iv + (ev // 4)) * level) // 100 + 5) * nature_modifier)

# Function to estimate IVs, EVs, and Nature for all stats
def estimate_all_stats(base_stats, observed_stats, level):
    possible_results = []

    # Iterate over all possible IVs (0-31) and EVs (0-252, step by 4)
    for ivs in itertools.product(range(32), repeat=5):
        for evs in itertools.product(range(0, 253, 4), repeat=5):
            for nature, modifiers in natures.items():
                matched = True
                for stat_key, base_stat in base_stats.items():
                    observed_stat = observed_stats[stat_key]
                    iv = ivs[list(base_stats.keys()).index(stat_key)]
                    ev = evs[list(base_stats.keys()).index(stat_key)]
                    nature_modifier = modifiers[stat_key]
                    calculated_stat = calculate_stat(base_stat, iv, ev, level, nature_modifier)

                    if calculated_stat != observed_stat:
                        matched = False
                        break

                if matched:
                    possible_results.append({
                        'ivs': ivs,
                        'evs': evs,
                        'nature': nature
                    })

    return possible_results

# Example Usage
# Assuming Haunter's base stats in Gen 8 are as follows:
base_stats = {
    'atk': 50,
}

# Observed stats at level 50
observed_stats = {
    'atk': 70,
}

level = 50

# Estimate possible matches
possible_matches = estimate_all_stats(base_stats, observed_stats, level)

# Print possible matches
for match in possible_matches:
    ivs = match['ivs']
    evs = match['evs']
    nature = match['nature']
    print(f"IVs: {ivs}, EVs: {evs}, Nature: {nature}")


KeyboardInterrupt: 

In [ ]:
def run_javascript(file_path, *args):
    result = subprocess.run(["node ", file_path] + list(args), capture_output=True, text=True, check=True)
    return result.stdout

# run_javascript("test.js", )
damage_calculator_js = "damage_calculation.js"
attacker = "Gengar"
defender = "Chansey"
move = "Thunderbolt"

damage = run_javascript(damage_calculator_js, attacker, defender, move)

if damage:
    print("Calculated damage:", damage)
    average_damage = sum(damage)/len(damage)
    print(f"Average damage: {average_damage}")
else:
    print("Could not calculate damage.")


In [ ]:
arg1 = 'Gengar'
arg2 = '100'
arg3 = 'leftovers'
arg4 = 'None'
arg5 = 'noability'
arg6 = "{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}"

arg7 = 'Chansey'
arg8 = '100'
arg9 = 'leftovers'
arg10 = 'None'
arg11 = 'noability'
arg12 = "{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}"

arg13 = 'Thunderbolt'

# damage_calc = run_javascript("damage_calculation.js", 
# attacker_name, attacker_level, attacker_item, attacker_status, attacker_ability, attacker_boosts, 
# defender_name, defender_level, defender_item, defender_status, defender_ability, defender_boosts, 
#                str(move))

damage_calc = run_javascript("damage_calculation.js", arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9, arg10, arg11, arg12, arg13)
print(damage_calc)

[79, 80, 80, 81, 81, 81, 82, 82, 82, 83, 83, 84, 84, 84, 85, 85, 85, 86, 86, 86, 87, 87, 88, 88, 88, 89, 89, 89, 90, 90, 91, 91, 91, 92, 92, 92, 93, 93, 94]
